In [1]:
import math as m
import numpy as np
import random as rand 

from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector

# Use Aer's qasm_simulator
simulator = QasmSimulator()

def W_state(q):
    # Create a Quantum Circuit acting on the q register
    circuit = QuantumCircuit(q,q)

    circuit.ry(2*np.arccos(1/m.sqrt(q)), 0)
    circuit.cry(2*np.arccos(1/m.sqrt(q-1)), 0, 1)
    circuit.ch(1, 2)
    circuit.cx(2, 3)
    circuit.cx(1, 2)
    circuit.cx(0, 1)
    circuit.x(0)

    psi = Statevector.from_instruction(circuit)
    # Probabilities for measuring qubits
    probs = psi.probabilities_dict()
    print('probs: {}'.format(probs))
   
    return circuit

In [13]:
def localPaulimeasurement(circuit,measurement_scheme):
    model_circuit = circuit
    for qubit_ind,qubit_measurement in enumerate(measurement_scheme):
        print(qubit_measurement, qubit_ind)
        if qubit_measurement == 'X':
            x_measurement(model_circuit,qubit_ind,qubit_ind)
        elif qubit_measurement == 'Y':
            y_measurement(model_circuit,qubit_ind,qubit_ind)
        else:
            model_circuit.measure(qubit_ind,qubit_ind)
    
    return model_circuit


In [18]:
from XandYmeasurements import *
from Measurement_schemes import *

circuit_size = 4
measurement_list = randomized_localPauli(circuit_size)
print(measurement_list[0])


for measurement_scheme in measurement_list:
    circuit = W_state(circuit_size)  
    #print(measurement_scheme,np.shape(measurement_scheme))
    model = localPaulimeasurement(circuit,measurement_scheme)
    
    # Compile the circuit down to low-level QASM instructions
    # supported by the backend
    compiled_circuit = transpile(model,simulator)

    # Execute the circuit on the qasm simulator
    job = simulator.run(compiled_circuit, shots=1)

    #Grab results from the job
    results = job.result().get_counts()
    print(results)
    print(model)


['X' 'X' 'X' 'X']
probs: {'0001': 0.2499999999999999, '0010': 0.2500000000000001, '0100': 0.25, '1000': 0.25}
X 0
X 1
X 2
X 3
{'0001': 1}
     ┌──────────┐                                  ┌───┐┌───┐┌─┐
q_0: ┤ Ry(2π/3) ├──────■────────────────────────■──┤ X ├┤ H ├┤M├
     └──────────┘┌─────┴──────┐               ┌─┴─┐├───┤└┬─┬┘└╥┘
q_1: ────────────┤ Ry(1.9106) ├──■─────────■──┤ X ├┤ H ├─┤M├──╫─
                 └────────────┘┌─┴─┐     ┌─┴─┐├───┤└┬─┬┘ └╥┘  ║ 
q_2: ──────────────────────────┤ H ├──■──┤ X ├┤ H ├─┤M├───╫───╫─
                               └───┘┌─┴─┐├───┤└┬─┬┘ └╥┘   ║   ║ 
q_3: ───────────────────────────────┤ X ├┤ H ├─┤M├───╫────╫───╫─
                                    └───┘└───┘ └╥┘   ║    ║   ║ 
c: 4/═══════════════════════════════════════════╩════╩════╩═══╩═
                                                3    2    1   0 
probs: {'0001': 0.2499999999999999, '0010': 0.2500000000000001, '0100': 0.25, '1000': 0.25}
X 0
X 1
X 2
Y 3
{'0000': 1}
     ┌──────────┐          